<p style="font-size:32px;text-align:center"> <b>Social network Graph Link Prediction - Facebook Challenge</b> </p>

In [0]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
train_graph=nx.read_edgelist("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /1.FB_EDA/train_pos_after_eda.csv",delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


# 2. Similarity measures

## 2.1 Jaccard Distance:
http://www.statisticshowto.com/jaccard-index/

Construct set of followers. Example followers of U1 and followers of U2. Say U1 = {U5,U3,U4}, U2 = {U3,U4,U6}

Since U1 and U2 have common followers so very high chance U1 and U2 will follow each other. 

So Jaccard distance = common folloewrs/total followers. So in above example Jaccard distance  = 2/4,it can be used in sets as well other than graph theory. 

I can do the samer thing for followee also like I did for followers. 


\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

In [0]:
def jaccard_for_followees(a,b):
  try:
    if len(set(train_graph.successors(a)))== 0 | len(set(train_graph.successors(b)))==0:
      return 0;
    sim= (len(set(train_graph.successors(a)).intersection(set(tarin_graph.successors(b)))))/\
    (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    
  except:
    return 0;
  
  return sim;


In [5]:
print(jaccard_for_followees(273084,1505602))

0


In [0]:
#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [7]:
print(jaccard_for_followers(273084,1505602))

0


In [8]:
print(jaccard_for_followers(273084,470294))

0



## 2.2 Cosine distance


We did learn cosine distance in two vectors. Instead of vector, if we have set.. like example above of graph.

if we have more overlap in sets, Cosine dist will be high. very similar to jaccard distance.  



\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [0]:
#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0

In [10]:
print(cosine_for_followees(273084,1505602))

0.0


In [0]:
def cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

In [12]:
print(cosine_for_followers(2,470294))

0.02886751345948129


In [13]:
print(cosine_for_followers(669354,1635354))

0


## 3. Ranking Measures

Very popular algo, it was used by good for ranking. Web page will have large value if lots of links are linked to it in the sense lots of web ages recieve links from that webpage. So would have higher value.  It gives direct graph. each vertex UI will give score to show how important the vertex is. Page rank will give reloave score of each vertex. so user has lot of  followers
so that vertex will have higher value.  Like Bill gates  has high followers so that vertex will have high value.

it is probabilkity value that random user will end up on this web page. 



https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.

<img src='PageRanks-Example.jpg'/>

Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. <b>(The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.</b>

## 3.1 Page Ranking

https://en.wikipedia.org/wiki/PageRank


In [0]:
if not os.path.isfile("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/page_rank.p"):
    pr = nx.pagerank(train_graph, alpha=0.85)
    pickle.dump(pr,open("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/page_rank.p",'wb'))
else:
    pr = pickle.load(open("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/page_rank.p",'rb'))

In [15]:
# values calculated for each vertex.

print('min',pr[min(pr, key=pr.get)])
print('max',pr[max(pr, key=pr.get)])
print('mean',float(sum(pr.values())) / len(pr))

min 1.6556497245737814e-07
max 2.7098251341935827e-05
mean 5.615699699389075e-07


In [16]:
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)

5.615699699389075e-07


# 4. Other Graph Features

## 4.1 Shortest path:

Getting Shortest path between twoo nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path. 

In [0]:
#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [18]:
compute_shortest_path_length(77697, 826021)

10

In [19]:
compute_shortest_path_length(669354,1635354)

-1

## 4.2 Checking for same community

In [0]:

wcc = list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
  index = []
  if train_graph.has_edge(b,a):
    return 1
  if train_graph.has_edge(a,b):
    for i in wcc:
      if a in i:
        index = i
        break
    if b in index:
      train_graph.remove_edge(a,b)
      if compute_shortest_path_length(a,b)== -1:
        train_graph.add_edge(a,b)
        return 0
      else:
        train_graph.add_edge(a,b)
        return 1
    else:
      return 0
  
  else:
    for i in wcc:
      if a in i:
        index = i
        break
    if b in index:
      return 1
    else:
      return 0


In [21]:
belongs_to_same_wcc(861, 1659750)

0

In [22]:
belongs_to_same_wcc(669354,1635354)

0

## 4.3 Adamic/Adar Index:
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [0]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        # common area of followees
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

In [24]:
calc_adar_in(1,189226)

0

## 4.4 Is persion was following back:

In [0]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [26]:
follows_back(1,189226)

1

## 4.5 Katz Centrality:
https://en.wikipedia.org/wiki/Katz_centrality

https://www.geeksforgeeks.org/katz-centrality-centrality-measure/
 Katz centrality computes the centrality for a node 
    based on the centrality of its neighbors. It is a 
    generalization of the eigenvector centrality. The
    Katz centrality for node `i` is
 
$$x_i = \alpha \sum_{j} A_{ij} x_j + \beta,$$
where `A` is the adjacency matrix of the graph G 
with eigenvalues $$\lambda$$.

The parameter $$\beta$$ controls the initial centrality and 

$$\alpha < \frac{1}{\lambda_{max}}.$$

In [0]:

if not os.path.isfile("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/katz.p"):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/katz.p",'wb'))
else:
    katz = pickle.load(open("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/atz.p",'rb'))

In [30]:
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0007313532484065916
max 0.003394554981699122
mean 0.0007483800935562018


## 4.6 Hits Score
The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.

https://en.wikipedia.org/wiki/HITS_algorithm

In [0]:
if not os.path.isfile("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/hits.p"):
    hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/hits.p",'wb'))
else:
    hits = pickle.load(open("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /2.Featurization/hits.p",'rb'))

In [36]:
print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


# 5. Featurization

## 5. 1 Reading a sample of Data from both train and test

In [0]:
import random
if os.path.isfile("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /1.FB_EDA/train_after_eda.csv"):
    filename = "/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /1.FB_EDA/train_after_eda.csv"
    f = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CS 3: Facebook Friend /1.FB_EDA/train_after_eda.csv", names=['source_node', 'destination_node'])
    f.shape
    # you uncomment this line, if you dont know the lentgh of the file name
    # here we have hardcoded the number of lines as 15100030
    # n_train = sum(1 for line in open(filename)) #number of records in file (excludes header)
    #n_train =  15100028
    #s = 100000 #desired sample size
    #skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))
    #https://stackoverflow.com/a/22259008/4084039